# The Enigma Machine

This cipher machine wase used during WWII and uses a set of transformations on each letter, which are updated for each letter, leading to a polyalphabetic cipher.

I have implemented parts of the machine in enigma.py and here I will demo it. Some parts of the machine I did not have time to model, so this will not behave as a true Enigma simulator, but it can still encrypt messages!

In [148]:
import enigma as en
import numpy as np

The process begins with a letter being pressed on the keyboard of the machine. This sends an electrical signal through the Plugboard (not implemnted), and then through the rotors. The Enigma machine usually had 3 rotors, but the model for submarines had 5!

As the signal passed through the rotor it was diverted to a different position, like a substitution. Therefore one letter as substituted three times as it passed through the rotors. 

Next in the circuit was the reflector, which performed pairwise substitution on all possible letters and sent the signal back through the rotors in reverse order, transforming them yet again!

The circuit was then complete and lit up a light under the encoded letter. This letter was noted, and as the key was depressed the rotors moved position (almost like an odometer (mine is simplified to be exactly like an odometer)) so that there would be a whole new set of substitutions for the next letter of the message.

This means that subsequent identical letters like 'aa' could be encoded as 'ud', because each letter had a different encryption.

There where several rotors and reflectors to chose from, all of which had different transformations and pairwise reflections respectively. To decode the message one simply had to set ones machine to exactly the same rotors and reflectors, in the same starting positions, and type in the code. Because of the reflector it would simply decode by using the machine twice!

## Implementation details
I treated each transformation as a matrix T (26 by 26, the number of letters on the machine squared) acting on a vector x (column vector of lenth 26) with a 1 in the position of the letter it represented.

(x[0] = 1) corresponds to 'a'

The transformation shifts the rows around so that the 1 moves to different letter positions.

## Cyclic permutatiotns for Rotors

The rotors where not in simple alphabetical order, each position in the alphabet has the possibility of being swapped for another. For the 5 normal rotors, labeled by roman numerals, this was done using several cyclic permutations

See definition of rotor1_cycles bellow: 'a' will be transformed to 'e', 'e' to 'l' and so on, with 'u' going to 'a'. In addition to these transformations, several other cycles were on each rotor, but some letters (like 's' for rotor 1) is transformed to itself.

Below I Create Rotor I-V, using permutations found on 
https://www.codesandciphers.org.uk/enigma/rotorspec.htm


In [237]:
# Creating Rotor I-V, using permutations found on 
# https://www.codesandciphers.org.uk/enigma/rotorspec.htm
rotor1_cycles = ['aeltphqxru','bknw', 'cmoy', 'dfg', 'iv', 'jz']
rotor2_cycles = ['fixvyomw', 'cdklhup', 'esz', 'bj', 'gr', 'nt']
rotor3_cycles = ['abdhpejt', 'cflvmzoyqirwukxsg']
rotor4_cycles = ['aepliywcoxmrfzbstgjqnh', 'dv', 'ku']
rotor5_cycles = ['avoldrwfiuq', 'bzksmnhyc', 'egtjpx']

t1 = en.list_cyclic_permutation_matrix(rotor1_cycles)
t2 = en.list_cyclic_permutation_matrix(rotor2_cycles)
t3 = en.list_cyclic_permutation_matrix(rotor3_cycles)
t4 = en.list_cyclic_permutation_matrix(rotor4_cycles)
t5 = en.list_cyclic_permutation_matrix(rotor5_cycles)


Here I create the rotor objects, with the permutation matrix. The second parameter is the inital position of the rotor in the machine (0 to 25) and the last parameter represents the notch. When this position is reached it causes the next rotor in line to rotate as well.

Feel free to play with the starting parameters!

The rotor in the first position was rotated every keypress, but the second one only rotates when the first one reaches its notch, and similar for two and three. In the real machine there was so called double-stepping in the second wheel, which I did not have time to implement.

In [238]:
rotor1 = en.Rotor(t1,0,17)
rotor2 = en.Rotor(t2,0,5)
rotor3 = en.Rotor(t3,0,22)
rotor4 = en.Rotor(t4,0,10)
rotor5 = en.Rotor(t5,0,0)

## Reflector

The reflector the substitution is pairwise

For reflector B (here called 1):
(AY) (BR) (CU) (DH) (EQ) (FS) (GL) (IP) (JX) (KN) (MO) (TZ) (VW)
are connected.

In [239]:
reload(en)
perms1 = ['ay', 'br', 'cu', 'dh', 'eq', 'fs', 'gl', 
         'ip', 'jx', 'kn', 'mo', 'tz', 'vw']
perms2 = ['af', 'bv', 'cp', 'dj', 'ei', 'go', 'hy', 
          'kr', 'lz', 'mx', 'nw', 'tq', 'su']
reflector1 = en.Reflector(perms1)
reflector2 = en.Reflector(perms2)


After the reflector the letter is transformed backwards through the rotors again, using the inverse of each rotors respective transformation. Since they are permutation matrices, the inverted form is simply the transposed version of the matrix.

## Machine
This class performs the total transformation. When it is reset, you can transform the message again, and it will be decoded!

In [240]:
machine = en.Machine([rotor1,rotor2,rotor3],reflector1)
# Only a-z, no punctuation
mess = en.Message("advancedscientificpythonprogramming")
machine.transformMessage(mess)
print(mess)
machine.reset()
machine.transformMessage(mess)
print(mess)

nqcxwofrmvuhsnpwhlxrnqxcjwqnscufdfb
advancedscientificpythonprogramming


## Where to go from here
If I had more time I could definitely implement the plugboard, which performed pairwise substitution before the rotors and where controlled by connecting two letters with a cable on the board.

The double-stepping behaviour of the rotors is more dificult to understand, but I could probably do it with more time.

Even further, it would be nice if it was easier to use for everyone, if you did not have to understand the code base to use the package, which is kind of the case now.

In [ ]:
## Thanks for the course